In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
# Activation function Relu
act = tf.nn.relu

In [3]:
# covolutional block => conv+ ReLU+ BatchNormalization
def dense_conv_block(input, feature_filter,filter_kernel,filter_kernel1,stride):
    
    x_in = keras.layers.Conv2D(feature_filter, filter_kernel, strides =stride, padding='same', activation=act)(input)
    x_in = keras.layers.BatchNormalization()(x_in)

    x = keras.layers.Conv2D(feature_filter, filter_kernel1, padding='same', activation=act)(x_in)
    x = keras.layers.Conv2D(feature_filter, filter_kernel1, padding='same', activation=act)(x)
    x = keras.layers.BatchNormalization( )(x)
    x = keras.layers.add([x, x_in])
    return x

In [ ]:
def build_model_128():
 
    image_in = keras.Input(shape=(128, 128, 1))
    Dconv_layer1 = dense_conv_block(image_in, 32,3,3,1)
    Dconv_layer2 = dense_conv_block(Dconv_layer1, 64,2,3,2)
    Dconv_layer3 = dense_conv_block(Dconv_layer2, 128,2,3,2)
    Dconv_layer4 = dense_conv_block(Dconv_layer3, 256,2,3,2)   
    # out 16 x 16 @256

    Uconv_block4 = keras.layers.Conv2DTranspose(128, 2, 2, activation=act)(Dconv_layer4)
    Uconv_block4 = keras.layers.BatchNormalization()(Uconv_block4)
    # out  32x32 @ 128

    # skip connection
    Uconv_block4 = keras.layers.concatenate([Dconv_layer3, Uconv_block4])
    Uconv_layer4 = dense_conv_block(Uconv_block4, 128,3,3,1)  
    # out 8 x 8 @ 128

    Uconv_block3 = keras.layers.Conv2DTranspose(64, 2, 2, activation=act)(Uconv_layer4)
    Uconv_block3 = keras.layers.BatchNormalization()(Uconv_block3)
    # out 16 x 16 @ 32

    # skip connection
    Uconv_block3 = keras.layers.concatenate([Uconv_block3, Dconv_layer2])
    Uconv_layer3 = dense_conv_block(Uconv_block3, 64,3,3,1)
    # out 16 x 16 @ 32

    Uconv_block2 = keras.layers.Conv2DTranspose(32, 2, 2, activation=act)(Uconv_layer3)
    Uconv_block2 = keras.layers.BatchNormalization()(Uconv_block2)
    Uconv_block2 = keras.layers.concatenate([Uconv_block2, Dconv_layer1])
     # skip connection
    Uconv_layer2 = dense_conv_block(Uconv_block2, 32,3,3)
    # out = 32 x 32 @ 64

    # 1 x 1 conv
    Uconv_layer1 = keras.layers.Conv2D(1, 3, padding='same', activation=keras.activations.sigmoid)(Uconv_layer2)
    model = keras.Model(image_in, Uconv_layer1)

    return model